# Model deployment

In this notebook, we will deploy the binary classification model created in the [previous notebook](train_model.ipynb) to a real-time AWS SageMaker endpoint. We will then use the model to make predictions on a test dataset. Please refer to the SageMaker Extension <a href="https://github.com/exasol/sagemaker-extension/blob/main/doc/user_guide/user_guide.md#prediction-on-aws-sagemaker-endpoint" target="_blank" rel="noopener">User Guide</a> for a detailed description of this process.

<b>Important! Please make sure you perform the last step - deletion of the endpoint. Leaving the endpoint in the cloud will incur continuous charges by AWS.</b>

We will be running SQL queries using <a href="https://jupysql.ploomber.io/en/latest/quick-start.html" target="_blank" rel="noopener"> JupySQL</a> SQL Magic.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the sandbox](../sandbox_config.ipynb).
2. [Initialize the SageMaker Extension](sme_init.ipynb).
3. [Load the MAGIC Gamma Telescope data](../data/data_telescope.ipynb).
4. [Train the model using SageMaker Autopilot](sme_train_model.ipynb).

## Setup

### Access configuration

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Let's bring up JupySQL and connect to the database via SQLAlchemy. Please refer to the documentation in the sqlalchemy-exasol for details on how to connect to the database using Exasol SQLAlchemy driver.

In [ ]:
%run ../utils/jupysql_init.ipynb

## Deploy model to a SageMaker endpoint

The script below deploys the best candidate model of the trained Autopilot job to an endpoint with a specified name. The deployment SQL command additionally generates the prediction UDF script with the same name. This UDF can be used for making predictions in an SQL statement.

<img src="utils/sme_deployment.png"/>
<center>Model deployment</center>

Let's define some variables.

In [ ]:
# Endpoint name, also the name of the generated UDF script.
ENDPOINT_NAME = "APSPredictor"

# The EC2 instance type of the endpoint to deploy the Autopilot model to.
INSTANCE_TYPE = "ml.m5.large"

# The initial number of instances to run the endpoint on.
INSTANCE_COUNT = 1

# Name of the table with the test data
TEST_TABLE_NAME = "TELESCOPE_TEST"

# Name of the column in the test table which is the prediction target.
TARGET_COLUMN = "CLASS"

In [ ]:
%%sql
EXECUTE SCRIPT {{sb_config.SCHEMA}}."SME_DEPLOY_SAGEMAKER_AUTOPILOT_ENDPOINT"(
    '{{sb_config.JOB_NAME}}', 
    '{{ENDPOINT_NAME}}', 
    '{{sb_config.SCHEMA}}',
    '{{INSTANCE_TYPE}}',  
    {{INSTANCE_COUNT}}, 
    '{{sb_config.SME_AWS_CONN}}', 
    '{{sb_config.AWS_REGION}}'
);

Let's check if the script has been created. We should be able to see an entry with the same name as the endpoint in the list of UDF scripts.

In [ ]:
%%sql
SELECT SCRIPT_NAME, SCRIPT_TYPE 
FROM SYS.EXA_ALL_SCRIPTS
WHERE SCRIPT_SCHEMA='{{sb_config.SCHEMA}}' AND SCRIPT_TYPE = 'UDF'

## Make predictions via SageMaker endpoint

Let's use the generated UDF script for making predictions on the test data.

First, we need to get a list of features to be passed to the UDF.

In [ ]:
%%sql column_names <<
SELECT COLUMN_NAME
FROM SYS.EXA_ALL_COLUMNS
WHERE COLUMN_SCHEMA = '{{sb_config.SCHEMA}}' AND COLUMN_TABLE='{{TEST_TABLE_NAME}}' AND COLUMN_NAME <> UPPER('{{TARGET_COLUMN}}');

In [ ]:
column_names = ', '.join(f'[{name[0]}]' for name in column_names)

Let's predict classes for the first 10 rows of the test table, just to see how the output of the UDF looks like. Remember that the first column in the input is reserved for the sample ID. Here we can just set it to zero.

In [ ]:
%%sql
SELECT "{{sb_config.SCHEMA}}"."{{ENDPOINT_NAME}}"(0, {{column_names}})
FROM "{{sb_config.SCHEMA}}"."{{TEST_TABLE_NAME}}"
LIMIT 10

Now we will compute the confusion matrix for the test data.

In [ ]:
%%sql
WITH TEST_DATA AS
(
    -- We take data from the test table and add the row number calling it SAMPLE_ID.
    SELECT ROW_NUMBER() OVER () AS SAMPLE_ID, {{column_names}}, [{{TARGET_COLUMN}}] FROM "{{sb_config.SCHEMA}}"."{{TEST_TABLE_NAME}}"
)
WITH MODEL_OUTPUT AS
(
    -- Make predictions. We will pass the SAMPLE_ID that sould be returned back unchanged.
    SELECT "{{sb_config.SCHEMA}}"."{{ENDPOINT_NAME}}"(SAMPLE_ID, {{column_names}})
    FROM TEST_DATA
)
-- Finally, compute the confusion matrix.
SELECT predictions, [{{TARGET_COLUMN}}], COUNT(*) as count
FROM MODEL_OUTPUT INNER JOIN TEST_DATA ON MODEL_OUTPUT.SAMPLE_ID = TEST_DATA.SAMPLE_ID
GROUP BY 1, 2;

## Delete endpoint

<b>It is important to delete the endpoint once we finished working with it, to avoid unnecessary charges.</b> The following script does that.

In [ ]:
%%sql
EXECUTE SCRIPT SME_DELETE_SAGEMAKER_AUTOPILOT_ENDPOINT(
    '{{ENDPOINT_NAME}}', 
    '{{sb_config.SME_AWS_CONN}}', 
    '{{sb_config.AWS_REGION}}'
)

## Conclusion

In this set of notebooks, we went through the steps required to train, deploy and use models based on the SageMaker Autopilot with the help of the Exasol SageMaker-Extension. The advantages the SageMaker-Extension provides include simple and fast uploading of training data into S3 buckets and getting predictions with SQL queries.